[View in Colaboratory](https://colab.research.google.com/github/rabex028/machine_learning_skynet/blob/master/spamNeuralNetwork.ipynb)

##Setting up imports (including dataset)

In [2]:
import sys
import tensorflow as tf
import numpy as np
import argparse
import os
import shutil
import pandas as pd
!pip install gspread
import gspread
import io
from sklearn.model_selection import train_test_split


#Package for neural networks
!pip install -q keras
import keras
from keras.models import Sequential
from keras.layers import Dense

from google.colab import files
uploaded = files.upload()

spambase = pd.read_csv(io.StringIO(uploaded['spambase.csv'].decode('utf-8')), header = None)
spambase

Using TensorFlow backend.


Saving spambase.csv to spambase.csv


,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.000,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.000,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.000,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.000,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.000,0.135,0.000,0.000,3.537,40,191,1
5,0.00,0.00,0.00,0.0,1.85,0.00,0.00,1.85,0.00,0.00,...,0.000,0.223,0.000,0.000,0.000,0.000,3.000,15,54,1
6,0.00,0.00,0.00,0.0,1.92,0.00,0.00,0.00,0.00,0.64,...,0.000,0.054,0.000,0.164,0.054,0.000,1.671,4,112,1
7,0.00,0.00,0.00,0.0,1.88,0.00,0.00,1.88,0.00,0.00,...,0.000,0.206,0.000,0.000,0.000,0.000,2.450,11,49,1
8,0.15,0.00,0.46,0.0,0.61,0.00,0.30,0.00,0.92,0.76,...,0.000,0.271,0.000,0.181,0.203,0.022,9.744,445,1257,1
9,0.06,0.12,0.77,0.0,0.19,0.32,0.38,0.00,0.06,0.00,...,0.040,0.030,0.000,0.244,0.081,0.000,1.729,43,749,1


##Initial Hypothesis

It doesn't look like there are hard and fast rules as to how many nodes should be in each layer, or how many layers there are, but a common starting point is an input layer with the number of features in each example, and the first hidden layer being half this amount. So our initial results will likely involve an input layer defined with 57 nodes to match the number of our features, and first hidden layer with 28 or so nodes. I think it might be better for our hidden layers to have a progressively smaller amount of nodes per level, which would mean that there will need to be aggregation of the results in previous layers. It might make sense then to go through the list of features and consider which ones may indicate an email is more likely to be spam and not spam (or neutral) and we can group our layers and nodes that way, and the result will be more effective.

##Creating and Running the Network (TensorFlow)

In [3]:
# Seperates independent and dependent variables
X = spambase.iloc[:,0:57].values
Y = spambase.iloc[:,57].values

# Uses sklearn module
# Creates training and test sets randomly with test set 20% of the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

# Defining Architecture 
# Only one hidden layer (for now)
def multilayer_perceptron(x, weights, biases, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    #layer_2 = tf.add(tf.matmul(x, weights['h2']), biases['b2'])
    #layer_2 = tf.nn.relu(layer_2)
    #layer_2 = tf.nn.dropout(layer_2, keep_prob)
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer
  
# Defining amounts of nodes, number of classifications
n_hidden_1 = 28
n_hidden_2 = 28
n_input = X_train.shape[1]
n_classes = 2

# Sets up weights and biases (connects nodes)
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    #'h2': tf.Variable(tf.random_normal([n_input, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    #'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

keep_prob = tf.placeholder("float")

# 5000 training optimization runs, results every 1000, 32 rows loaded in each time
training_epochs = 5000
display_step = 1000
batch_size = 32

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder(tf.int64, [None])

predictions = multilayer_perceptron(x, weights, biases, keep_prob)

#reg_constant = 20
#reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=predictions))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(X_train) / batch_size)
        x_batches = np.array_split(X_train, total_batch)
        y_batches = np.array_split(Y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: 0.8
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: X_test, y: Y_test, keep_prob: 1.0})
    
    #Printing aggregated Metrics from output layer
    print(predictions.eval(feed_dict = {x: X_test, y: Y_test, keep_prob: 1.0}))
    
    #Printing vector with predicted labels
    print(sess.run(tf.argmax(predictions,1), {x: X_test, y: Y_test, keep_prob: 1.0}))
    
    #Printing vector with actual labels
    print(Y_test)
    
    #Defining and printing confusion Matrix
    cm = tf.contrib.metrics.confusion_matrix(Y_test, pred)
    with tf.Session() as sess:
      sess.run(cm)
      print(sess.run(cm))
 

Epoch: 0001 cost= 375.033540543


KeyboardInterrupt: ignored

##Creating and Running the Network (Keras)

In [6]:
# Initializing the Neural Network
spamClassifier = Sequential()

# Defining input layer and first hidden layer
# Number of nodes matches features, features/2
spamClassifier.add(Dense(units = 28, kernel_initializer = 'uniform', activation = 'relu', input_dim = 57))

# Defining 2nd layer
spamClassifier.add(Dense(units = 28, kernel_initializer = 'uniform', activation = 'relu'))

# Defining output layer
spamClassifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Setting hyperparameters
spamClassifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

spamClassifier.fit(X_train, Y_train, batch_size = 20, epochs = 100)

Epoch 1/100
3680/3680 [==============================] - 1s 390us/step - loss: 0.5766 - acc: 0.7508
Epoch 2/100
3680/3680 [==============================] - 1s 349us/step - loss: 0.3975 - acc: 0.8448
Epoch 3/100
3680/3680 [==============================] - 1s 333us/step - loss: 0.3174 - acc: 0.8848
Epoch 4/100
3680/3680 [==============================] - 1s 342us/step - loss: 0.2735 - acc: 0.9027
Epoch 5/100
3680/3680 [==============================] - 1s 332us/step - loss: 0.2385 - acc: 0.9201
Epoch 6/100
3680/3680 [==============================] - 1s 342us/step - loss: 0.2366 - acc: 0.9163
Epoch 7/100
3680/3680 [==============================] - 1s 339us/step - loss: 0.2303 - acc: 0.9179
Epoch 8/100
3680/3680 [==============================] - 1s 330us/step - loss: 0.2147 - acc: 0.9190
Epoch 9/100
1840/3680 [==============>...............] - ETA: 0s - loss: 0.2616 - acc: 0.9082

3680/3680 [==============================] - 1s 341us/step - loss: 0.2369 - acc: 0.9147
Epoch 10/100
1740/3680 [=============>................] - ETA: 0s - loss: 0.1951 - acc: 0.9230

KeyboardInterrupt: ignored

##Resources:</br>
https://towardsdatascience.com/neural-networks-with-google-colaboratory-artificial-intelligence-getting-started-713b5eb07f14 </br>
https://medium.com/@curiousily/tensorflow-for-hackers-part-ii-building-simple-neural-network-2d6779d2f91b</br>
https://www.analyticsvidhya.com/blog/2016/10/an-introduction-to-implementing-neural-networks-using-tensorflow/